In [3]:
import nltk
nltk.download()
from nltk import word_tokenize, pos_tag, ne_chunk
import string
import warnings
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from wordcloud import WordCloud,STOPWORDS
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import RidgeClassifier,LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [4]:
df_train = pd.read_csv("../Data/train.csv")
df_game = pd.read_csv("../Data/game_overview.csv")
df_test = pd.read_csv("../Data/test.csv")


In [5]:
print(df_train.head())
print(df_test.head())


   review_id                        title    year  \
0          1  Spooky's Jump Scare Mansion  2016.0   
1          2  Spooky's Jump Scare Mansion  2016.0   
2          3  Spooky's Jump Scare Mansion  2016.0   
3          4  Spooky's Jump Scare Mansion  2015.0   
4          5  Spooky's Jump Scare Mansion  2015.0   

                                         user_review  user_suggestion  
0  I'm scared and hearing creepy voices.  So I'll...                1  
1  Best game, more better than Sam Pepper's YouTu...                1  
2  A littly iffy on the controls, but once you kn...                1  
3  Great game, fun and colorful and all that.A si...                1  
4  Not many games have the cute tag right next to...                1  
   review_id                             title    year  \
0       1603  Counter-Strike: Global Offensive  2015.0   
1       1604  Counter-Strike: Global Offensive  2018.0   
2       1605  Counter-Strike: Global Offensive  2018.0   
3       1606  Cou

In [6]:
print("Number of rows are", df_train.shape[0], ".Number of columns is", df_train.shape[1])
print("Number of rows are", df_test.shape[0], ".Number of columns is", df_test.shape[1])


Number of rows are 17494 .Number of columns is 5
Number of rows are 8045 .Number of columns is 4


In [7]:
# Data Type of each column.Return Object.
print("Dataframe data types")
print(df_train.dtypes)
print(df_test.dtypes)


Dataframe data types
review_id            int64
title               object
year               float64
user_review         object
user_suggestion      int64
dtype: object
review_id        int64
title           object
year           float64
user_review     object
dtype: object


In [8]:
# Return column names as Index object.
print("Dataframe column data types")
print(df_train.columns)
print(df_test.columns)


Dataframe column data types
Index(['review_id', 'title', 'year', 'user_review', 'user_suggestion'], dtype='object')
Index(['review_id', 'title', 'year', 'user_review'], dtype='object')


In [9]:
# Class distribution of target available.
print("Distribution of target variable")
print(df_train.user_suggestion.value_counts())


Distribution of target variable
1    9968
0    7526
Name: user_suggestion, dtype: int64


In [10]:
# Class distribution of title variable.
print("Distribution of title variable")
print(df_train.title.value_counts())


Distribution of title variable
Robocraft                                             842
Eternal Card Game                                     791
Heroes & Generals                                     745
War Thunder                                           720
Fractured Space                                       718
Bless Online                                          712
The Elder Scrolls®: Legends™                          565
Neverwinter                                           546
AdventureQuest 3D                                     519
theHunter Classic                                     518
Creativerse                                           492
DCS World Steam Edition                               488
Team Fortress 2                                       479
Infestation: The New Z                                479
PlanetSide 2                                          472
Path of Exile                                         458
SMITE®                                   

In [11]:
print("Converting all letters to lowercase")
# Convert text to lowercase.
def tolowercase(text):
    text = text.lower()
    return text

df_train['user_review'] = df_train.user_review.apply(tolowercase)
df_test['user_review'] = df_test.user_review.apply(tolowercase)

Converting all letters to lowercase


In [12]:
print("Removing punctuation")

# Remove punctuation.
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

df_train['user_review'] = df_train.user_review.apply(remove_punctuation)
df_test['user_review'] = df_test.user_review.apply(remove_punctuation)


Removing punctuation


In [13]:
print("Perform lemmatization")

# Lemmatization:
def do_lemmatization(text):
    lemma_words = set([])
    lemmatizer = WordNetLemmatizer()
    text = word_tokenize(text)
    for word in text:
        lemma_words.add(lemmatizer.lemmatize(word))
    return " ".join(lemma_words)

df_train['user_review'] = df_train.user_review.apply(do_lemmatization)
df_test['user_review'] = df_test.user_review.apply(do_lemmatization)


Perform lemmatization


In [14]:
print("Perform named entity recognition")

def named_entity_recognition(text):
    result = ne_chunk(pos_tag(word_tokenize(text)))
    return result

text_1 = df_train.user_review[0]
ner = named_entity_recognition(text_1)
print(ner)




Perform named entity recognition
(S
  music/NN
  review/NN
  pause/IN
  a/DT
  are/VBP
  childhood/JJ
  themor/JJ
  let/NN
  bubble/JJ
  around/IN
  room/NN
  like/IN
  if/IN
  noob/IN
  me/PRP
  were/VBD
  is/VBZ
  ghost/VBN
  to/TO
  with/IN
  adorable/JJ
  finding/VBG
  ill/NN
  of/IN
  from/IN
  locked/VBN
  such/JJ
  flashlight/JJ
  hearing/VBG
  thing/NN
  at/IN
  the/DT
  likable/JJ
  while/IN
  wait/NN
  im/VBZ
  more/JJR
  friend/JJ
  turn/NN
  see/VBP
  stand/NN
  scared/VBN
  full/JJ
  on/IN
  graphic/JJ
  moment/NN
  door/NN
  menever/NN
  charactes/VBZ
  game/NN
  odd/''
  so/RB
  look/VB
  though/IN
  afraid/JJ
  there/EX
  1990swhat/CD
  in/IN
  kill/NN
  asoh/NNS
  have/VBP
  before/IN
  staring/VBG
  but/CC
  whats/VBZ
  them/PRP
  sceme/JJ
  voice/NN
  dead/JJ
  i/NN
  did/VBD
  somewhat/RB
  shine/JJ
  beat/NN
  and/CC
  clean/JJ
  time/NN
  bit/NN
  few/JJ
  tree/NN
  chasing/VBG
  write/JJ
  return/NN
  that/WDT
  can/MD
  heart/NN
  this/DT
  hmm/NN
  hello/VBZ
  

In [15]:
print("Part-of-speech tagging using NLTK")

def pos_tagging(text):
    text = word_tokenize(text)
    tokens_tag = pos_tag(text)
    return tokens_tag

df_train['pos_tagging'] = df_train.user_review.apply(pos_tagging)
print(df_train['pos_tagging'])


Part-of-speech tagging using NLTK
0        [(music, NN), (review, NN), (pause, IN), (a, D...
1        [(game, NN), (sam, VB), (your, PRP$), (1010wha...
2        [(game, NN), (hilarity, NN), (control, NN), (e...
3        [(game, NN), (yeah, NN), (shame, NN), (cheer, ...
4        [(game, NN), (right, RB), (being, VBG), (and, ...
                               ...                        
17489    [(appearance, RB), (your, PRP$), (a, DT), (pay...
17490    [(your, PRP$), (right, JJ), (charge, NN), (own...
17491    [(anymore, RB), (friendly, JJ), (where, WRB), ...
17492    [(game, NN), (only, RB), (looping, VBG), (your...
17493    [(game, NN), (day, NN), (year, NN), (always, R...
Name: pos_tagging, Length: 17494, dtype: object


In [16]:
# NUMBER OF 15 MOST FREQUENT TERMS.
token = nltk.word_tokenize(''.join(df_train.user_review))
frequent = nltk.FreqDist(token)
print(frequent.most_common(15))


[('the', 15376), ('and', 14953), ('to', 14679), ('a', 14588), ('it', 13775), ('is', 12972), ('this', 12316), ('i', 11684), ('you', 11246), ('of', 10461), ('for', 10279), ('in', 10096), ('but', 9713), ('that', 9448), ('game', 8839)]


In [17]:
# Text with highest number of words.
df_train['number_of_words'] = df_train['user_review'].apply(lambda x: len(str(x).split()))
print('Maximum number of word',df_train['number_of_words'].max())
print('\nSentence:\n',df_train[df_train['number_of_words'] == 587]['user_review'].values)


Maximum number of word 587

Sentence:
 ['nika 1km your right meeting ryazan woodburning deceived kneeling 8th are shake weapon bird let those you not superpower euro tube 666ed worship ad long want with exhibit serve cover need instead planetary left blow costume crowbar longer at very correctly stick the prison except showing while portuguese stripper alexandre azazel enoch govt remain muslim administered nuke live plague 11th gaad scandinavia youre shoulder old leg 2006 german short rocket rejecting airplane destroy cross siberia satan created allegiance medical xc seraph wont ufo commonly lot say next lower there believe body water kill skin clergy between synthetic have side week go christ giant 7525 prayer sergiy blame bowing higher river stay participate america earthquake clone base betraying first inland fake return can sarov this during plate filling russia creation fast 1st put way shroud his month off third high forward bomb all either incorrectly clothfabrictextile their co

In [18]:
X = df_train['user_review']
y = df_train['user_suggestion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(13995,)
(3499,)
(13995,)
(3499,)


In [19]:
# # Feature Extraction using TFIDF-Char Based.
tfidf_vec = TfidfVectorizer(ngram_range=(1,5), stop_words='english', analyzer='char',max_features=5000)
print(type(tfidf_vec))  # TfidfVectorizer class.
train_tfidf_vec = tfidf_vec.fit_transform(X_train)
print(type(train_tfidf_vec))  # Sparse CSR matrix.
valid_tfidf_vec = tfidf_vec.transform(X_test)
print(type(valid_tfidf_vec))  # Sparse CSR matrix.
test_tfidf_vec = tfidf_vec.transform(df_test['user_review'])
train_vector_array = train_tfidf_vec.toarray()
valid_vector_array = valid_tfidf_vec.toarray()
test_vector_array = test_tfidf_vec.toarray()

print(train_vector_array.shape)
print(valid_vector_array.shape)
print(test_vector_array.shape)


<class 'sklearn.feature_extraction.text.TfidfVectorizer'>
<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
(13995, 5000)
(3499, 5000)
(8045, 5000)


In [20]:
# Linear support vector classifier.
lsvc = LinearSVC(C=1,loss= 'hinge',random_state=999)
lsvc.fit(train_vector_array, y_train)
y_pred = lsvc.predict(valid_vector_array)
print(f1_score(y_test,y_pred,average='micro'))   # 0.8250


0.8236639039725637


In [21]:
# Logistic Regression.
lr = LogisticRegression(random_state=999)
lr.fit(train_vector_array, y_train)
y_pred = lr.predict(valid_vector_array)
print(f1_score(y_test,y_pred,average='micro'))  # 0.8128




0.8142326378965419


In [22]:
# Random Forest Classifier.
rfc = RandomForestClassifier(n_estimators=300, random_state=999)
rfc.fit(train_vector_array, y_train)
pred = rfc.predict(valid_vector_array)
print(f1_score(y_test, pred))  # 0.8270



0.8320895522388061


In [23]:
# Bernoulli Naive Bayes.
bnb = BernoulliNB()
bnb.fit(train_vector_array,y_train)
y_pred = bnb.predict(valid_vector_array)
print(f1_score(y_test,y_pred,average='micro'))  # 0.6547


0.6524721348956845


In [24]:
# Ridge Classifier.
ridge = RidgeClassifier(random_state=999)
ridge.fit(train_vector_array,y_train)
y_pred = ridge.predict(valid_vector_array)
print(f1_score(y_test,y_pred,average='micro'))  # 0.8348


0.8296656187482138


In [25]:
pred_final = ridge.predict((test_vector_array))
df = pd.DataFrame({'review_id': df_test['review_id'], 'user_suggestion': pred_final})
df.to_csv("submission_results.csv")

In [26]:
#################################################
# LDA Topic Modelling:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(X_train)
print(doc_term_matrix)

  (0, 19392)	1
  (0, 15277)	1
  (0, 18618)	1
  (0, 45)	1
  (0, 5516)	1
  (0, 2358)	1
  (0, 2953)	1
  (0, 18389)	1
  (0, 796)	1
  (0, 8320)	1
  (0, 1325)	1
  (0, 18572)	1
  (0, 18426)	1
  (0, 13755)	1
  (0, 7996)	1
  (0, 8966)	1
  (0, 2816)	1
  (0, 18661)	1
  (0, 9297)	1
  (1, 12065)	1
  (1, 18452)	1
  (1, 12227)	1
  (1, 9062)	1
  (1, 7306)	1
  (1, 260)	1
  :	:
  (13994, 2848)	1
  (13994, 5619)	1
  (13994, 10803)	1
  (13994, 6974)	1
  (13994, 1532)	1
  (13994, 16951)	1
  (13994, 16978)	1
  (13994, 12286)	1
  (13994, 16455)	1
  (13994, 8247)	1
  (13994, 14371)	1
  (13994, 7565)	1
  (13994, 12703)	1
  (13994, 19458)	1
  (13994, 10333)	1
  (13994, 11531)	1
  (13994, 19408)	1
  (13994, 15822)	1
  (13994, 11806)	1
  (13994, 13418)	1
  (13994, 19409)	1
  (13994, 4395)	1
  (13994, 16924)	1
  (13994, 4024)	1
  (13994, 15697)	1


In [27]:
# Each of x documents is represented as y dimensional vector,which means that our vocabulary has y words.
from sklearn.decomposition import LatentDirichletAllocation
LDA = LatentDirichletAllocation(n_components=5,
                                random_state=0)
print(type(LDA))  # LatentDirichletAllocation class.
LDA.fit(doc_term_matrix)
print(LDA.components_)

<class 'sklearn.decomposition._lda.LatentDirichletAllocation'>
[[1.50392197 0.20285865 0.20000048 ... 0.20001073 0.20001242 0.20000421]
 [2.22030621 9.26054728 0.20059775 ... 0.2019053  1.19880946 4.19996877]
 [0.2029754  0.20583334 0.20000148 ... 0.20003713 0.2000419  0.20001373]
 [1.87005562 0.21013206 0.20395717 ... 2.19803437 1.20111946 0.20000869]
 [0.2027408  2.12062867 2.19544313 ... 0.20001246 0.20001676 0.20000461]]


In [28]:
# For each topic,each word of the document is assigned a weight.
# Higher weight means it is the top word of the topic.
# It is a multidimensional array.Each row represent the topic,each column represents the word in a document.
# Shape = [n_topics,n_words] or [n_components, n_features]

# Define helper function to print top words for each topic.
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        print(topic)
        message = "\nTopic #{}:".format(index)
        print(message)
        print([(feature_names[i], topic[i]) for i in topic.argsort()[:-n_top_words - 1:-1]])
        # feature_names[i] is a word,topic[i] is the weight of the word for that topic.
        print("=" * 70)

number_of_words = 50
print("\nTopics in LDA model: ")
tf_feature_names = count_vect.get_feature_names()
print_top_words(LDA, tf_feature_names, number_of_words)
#####################################################


Topics in LDA model: 
[1.50392197 0.20285865 0.20000048 ... 0.20001073 0.20001242 0.20000421]

Topic #0:
[('just', 1246.2293574901844), ('play', 1238.2028233219012), ('like', 1233.0199735780093), ('money', 1164.4577990595262), ('time', 1160.5062199639895), ('dont', 1113.8961971124868), ('pay', 982.8027046425142), ('fun', 982.7455095978125), ('free', 955.6708150453089), ('want', 938.622537226889), ('buy', 835.6175458444825), ('really', 808.7895968052751), ('make', 800.7320921994668), ('good', 777.9739181608707), ('thing', 761.289386036078), ('need', 755.7732242094686), ('playing', 714.0867502575835), ('real', 693.6838553295996), ('people', 682.554808900431), ('hour', 677.0993088320358), ('spend', 635.7362837314652), ('way', 617.0253866954583), ('lot', 612.7801510685374), ('player', 607.7477914293787), ('ha', 606.5678635337366), ('level', 577.049199801039), ('new', 560.4528272964587), ('item', 543.953513111264), ('win', 540.9006571508414), ('youre', 499.7361573092421), ('great', 474.192

In [29]:
######################################################
# Topic Modelling : NMF: Non-Matrix factorization.
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(X_train)
print(doc_term_matrix)

  (0, 9297)	0.3425246833049525
  (0, 18661)	0.08573368833073064
  (0, 2816)	0.14753762362275624
  (0, 8966)	0.27963845872269194
  (0, 7996)	0.33209505779521536
  (0, 13755)	0.09000125920816517
  (0, 18426)	0.33209505779521536
  (0, 18572)	0.17755586664389592
  (0, 1325)	0.3425246833049525
  (0, 8320)	0.2351464758651997
  (0, 796)	0.1606282261167631
  (0, 18389)	0.1335134519770323
  (0, 2953)	0.16127419418700661
  (0, 2358)	0.10471240034982547
  (0, 5516)	0.13709405694814777
  (0, 45)	0.17755586664389592
  (0, 18618)	0.2841761231572764
  (0, 15277)	0.29542047955612927
  (0, 19392)	0.20878805199845787
  (1, 16)	0.13852512083288648
  (1, 3842)	0.3337102176610715
  (1, 11877)	0.1448339864040923
  (1, 11804)	0.2733004351761694
  (1, 3265)	0.21869967632646073
  (1, 4451)	0.22503297592694574
  :	:
  (13994, 1532)	0.14086247789798925
  (13994, 6974)	0.1366547211499745
  (13994, 10803)	0.12911524767253468
  (13994, 5619)	0.18799404319370158
  (13994, 2848)	0.17367403228416955
  (13994, 1949)	0.

In [30]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=5, random_state=42)
print(type(nmf))
nmf.fit(doc_term_matrix)
print(nmf.components_)

<class 'sklearn.decomposition._nmf.NMF'>
[[0.00171902 0.0027099  0.00031854 ... 0.         0.         0.00199683]
 [0.         0.         0.00044252 ... 0.00990441 0.00315435 0.        ]
 [0.         0.00081488 0.         ... 0.         0.         0.        ]
 [0.         0.00186065 0.         ... 0.         0.         0.        ]
 [0.00225753 0.00293271 0.0002522  ... 0.         0.         0.        ]]


In [31]:
# # For each topic,each word of the document is assigned a weight.
# # Higher weight means it is the top word of the topic.
# # It is a multidimensional array.Each row represent the topic,each column represents the word in a document.
# Shape - [n_topics,n_words] or [n_components, n_features].
# Factorization matrix.

# Define helper function to print top words.
def print_top_words(model, feature_names, n_top_words):
    for index, topic in enumerate(model.components_):
        message = "\nTopic #{}:".format(index)
        print(message)
        print([(feature_names[i], topic[i]) for i in topic.argsort()[:-n_top_words - 1:-1]])
        print("=" * 70)

number_of_words = 50
print("\nTopics in NMF model: ")
tf_feature_names = tfidf_vect.get_feature_names()  # note that tf_vectorizer is an LemmaCountVectorizer object and with this command we get the whole dictionary of words
print_top_words(nmf, tf_feature_names, number_of_words)

######################################################



Topics in NMF model: 

Topic #0:
[('fun', 0.9281842024008672), ('play', 0.8083239228961154), ('great', 0.7745282107161623), ('really', 0.7469751606270683), ('good', 0.7215021616895377), ('like', 0.7168866091663099), ('free', 0.6476114353880471), ('recommend', 0.6410729028072932), ('friend', 0.6088296350687633), ('lot', 0.5761197880546383), ('best', 0.5495056632441343), ('graphic', 0.545972596998205), ('love', 0.5323032329703362), ('played', 0.5319209500998823), ('playing', 0.41864924084202176), ('pretty', 0.41519243811565515), ('try', 0.39260062975277016), ('amazing', 0.3870552553422343), ('nice', 0.37207212472533163), ('feel', 0.3640155034509318), ('ha', 0.363143984671539), ('gameplay', 0.332672896220805), ('ive', 0.3318293033970295), ('character', 0.3283686219637916), ('time', 0.3261920193971382), ('thing', 0.315745145384543), ('bit', 0.30415301691239577), ('better', 0.29865420205090637), ('overall', 0.2804461706572826), ('look', 0.27992350785384684), ('just', 0.27694606684479445), 